
##Ingest constructors.json file
Path : dbfs:/mnt/formula1dlsaga/raw/constructors.json

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date= dbutils.widgets.get("p_file_date")


In [0]:
constructors_schema = "constructorId INT NOT NULL, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
constructors_df = spark.read.schema(constructors_schema).json(f"{raw_folder_path}/{v_file_date}/constructors.json")

In [0]:
constructors_drop_df = constructors_df.drop("url")

In [0]:
constructors_final_df_func = constructors_drop_df.withColumnRenamed("constructorId", "constructor_id") \
                                             .withColumnRenamed("constructorRef", "constructor_ref")\
                                             .withColumn("data_source", lit(v_data_source))\
                                             .withColumn("file_date", lit(v_file_date)) 


In [0]:
constructors_final_df = add_ingestion_date(constructors_final_df_func)

In [0]:
# constructors_final_df.write.parquet(f"{processed_folder_path}/constructors", mode="overwrite")

In [0]:
constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
%sql
SELECT * FROM f1_processed.constructors;

In [0]:
# df = spark.read.parquet(f"{processed_folder_path}/constructors")
# display(df)

In [0]:
dbutils.notebook.exit("Success")